# Introduction

This notebook demonstrates how to utilise the calculate_profile_data method for x-ray diffraction simulations of different crystal structures. The structes demonstrated in this notebook are:
    -Fm-3d Silicon
    -Bcc Iron
    -Fcc Nickel
    -Graphite 
    -Arbitray simple cubic (with Ni atoms)

This functionaility has been checked to run in pyxem-0.11.0 (May 2020). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/pyxem-demos/issues

1. <a href='#tem'> Importing generators</a>
2. <a href='#vec'> Define Structures</a>
3. <a href='#vec'> Define the diffraction generator class</a>
4. <a href='#vec'> Plotting
5. <a href='#vec'> Index of methods

# <a id='tem'></a> 1. Import Generators

Import generators required for simulation and indexation

In [ ]:
import diffpy
import numpy as np
from math import pi
from transforms3d.euler import euler2mat
import collections
import matplotlib.pyplot as plt

from diffsims.sims.diffraction_simulation import DiffractionSimulation

from diffsims.utils.atomic_scattering_params import ATOMIC_SCATTERING_PARAMS
from diffsims.utils.lobato_scattering_params import ATOMIC_SCATTERING_PARAMS_LOBATO
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.sims.diffraction_simulation import  DiffractionSimulation

from diffsims.utils.atomic_diffraction_generator_support.fourier_transform import (
    from_recip,
)

# <a id='tem'></a> 2. Define structures

Define or import the structures to be used in this simulation. 

In [ ]:
def make_si_structure(lattice_parameter=None):
    """
    We construct an Fd-3m silicon (with lattice parameter 5.431 as a default)
    """
    if lattice_parameter is not None:
        a = lattice_parameter
    else:
        a = 5.431
    latt = diffpy.structure.lattice.Lattice(a, a, a, 90, 90, 90)
    atom_list = []
    for coords in [[0, 0, 0], [0.5, 0, 0.5], [0, 0.5, 0.5], [0.5, 0.5, 0]]:
        x, y, z = coords[0], coords[1], coords[2]
        atom_list.append(
            diffpy.structure.atom.Atom(atype="Si", xyz=[x, y, z], lattice=latt)
        )  # Motif part A
        atom_list.append(
            diffpy.structure.atom.Atom(
                atype="Si", xyz=[x + 0.25, y + 0.25, z + 0.25], lattice=latt
            )
        )  # Motif part B
    return diffpy.structure.Structure(atoms=atom_list, lattice=latt)

def make_fe_structure():
    a = 2.856
    latt = diffpy.structure.lattice.Lattice(a, a, a, 90, 90, 90)
    atom_list = []
    for coords in [[0, 0, 0], [0.5, 0.5, 0.5]]:
        x, y, z = coords[0], coords[1], coords[2]
        atom_list.append(
            diffpy.structure.atom.Atom(atype="Fe", xyz=[x, y, z], lattice=latt)
        )
    return diffpy.structure.Structure(atoms=atom_list, lattice=latt)

def make_ni_structure():
    a = 3.499
    latt = diffpy.structure.lattice.Lattice(a, a, a, 90, 90, 90)
    atom_list = []
    for coords in [[0, 0, 0], [0.5, 0, 0.5], [0, 0.5, 0.5], [0.5, 0.5, 0]]:
        x, y, z = coords[0], coords[1], coords[2]
        atom_list.append(
            diffpy.structure.atom.Atom(atype="Ni", xyz=[x, y, z], lattice=latt)
        )
    return diffpy.structure.Structure(atoms=atom_list, lattice=latt)

def make_graphite_structure():
    a = 2.461
    c = 6.708
    latt = diffpy.structure.lattice.Lattice(a, a, c, 90, 90, 120)
    atom_list = []
    coords = [0, 0, 0]
    x, y, z = coords[0], coords[1], coords[2]
    atom_list.append(
        diffpy.structure.atom.Atom(atype="C", xyz=[x, y, z], lattice=latt)
    )
    return diffpy.structure.Structure(atoms=atom_list, lattice=latt)

def make_cubic_structure():
    a = 3
    latt = diffpy.structure.lattice.Lattice(a, a, a, 90, 90, 90)
    atom_list = []
    coords = [0, 0, 0]
    x, y, z = coords[0], coords[1], coords[2]
    atom_list.append(
        diffpy.structure.atom.Atom(atype="Ni", xyz=[x, y, z], lattice=latt)
    )
    return diffpy.structure.Structure(atoms=atom_list, lattice=latt)


In [ ]:
Si_struct = make_si_structure()
Fe_struct = make_fe_structure()
Ni_struct = make_ni_structure()
graphite_struct = make_graphite_structure()
cubic_struct = make_cubic_structure()
wz_struct = diffpy.structure.loadStructure('/Users/Wood/Documents/GitHub/cif_files/GaAs_mp-8883_conventional_standard.cif')

# <a id='veca'></a> 3. Generate diffraction data

Using the calcualte_profile_data method from the DiffractionGenerator class 

Once defined can use the caluclate_profile_data method to produce the 

In [ ]:
diff_gen = DiffractionGenerator(accelerating_voltage=300,
                                max_excitation_error=1/10)

In [ ]:
si_prof = diff_gen.calculate_profile_data(
                        Si_struct,
                        reciprocal_radius=1.0,
                        magnitude_tolerance=1e-5,
                        minimum_intensity=1e-3,
                        )

fe_prof = diff_gen.calculate_profile_data(
                        Fe_struct,
                        reciprocal_radius=1.0,
                        magnitude_tolerance=1e-5,
                        minimum_intensity=1e-3,
                        )

ni_prof = diff_gen.calculate_profile_data(
                        Ni_struct,
                        reciprocal_radius=1.0,
                        magnitude_tolerance=1e-5,
                        minimum_intensity=1e-3,
                        )

graphite_prof = diff_gen.calculate_profile_data(
                        graphite_struct,
                        reciprocal_radius=1.0,
                        magnitude_tolerance=1e-5,
                        minimum_intensity=1e-3,
                        )

cubic_prof = diff_gen.calculate_profile_data(
                        cubic_struct,
                        reciprocal_radius=1.0,
                        magnitude_tolerance=1e-5,
                        minimum_intensity=1e-3,
                        )


# <a id='veca'></a> 4. Plotting


Using the get_plot method from the DiffractionSimulation class to visualise the data produced.

In [ ]:
si_plot = si_prof.get_plot(5)


In [ ]:
fe_plot = fe_prof.get_plot(5)

In [ ]:
ni_plot = ni_prof.get_plot(5)

In [ ]:
grpahite_plot = graphite_prof.get_plot(5)

In [ ]:
cubic_plot = cubic_prof.get_plot(5)

# <a id='veca'></a> 5. Index of methods


Libary of the importnant methods used in the above simulations.

In [ ]:
def get_intesnities_params_dict(reciprocal_lattice, reciprocal_radius):

    spot_indices, _, spot_distances = get_points_in_sphere(reciprocal_lattice, reciprocal_radius)
    
    dict_i_to_d = {}
    for i,d in zip(spot_indices, spot_distances):
        dict_i_to_d[tuple(i)] = d
    
    list_hkls = spot_indices.tolist()
    
    unique_hkls_dict = get_unique_families(list_hkls)
    
    multiplicites = np.fromiter(unique_hkls_dict.values(), dtype=float)
    unique_hkls1 = np.array(list(unique_hkls_dict))

    g_hkls = []
    for unique_hkl in unique_hkls1:
        g_hkls.append(dict_i_to_d[tuple(unique_hkl)])
        
    return unique_hkls1, multiplicites, g_hkls
    
    


In [ ]:
def get_kinematical_intensities(
    structure,
    g_indices,
    g_hkls_array,
    debye_waller_factors,
    multiplicites,
    scattering_params,
    shape_factor,
):

    """Calculates peak intensities.

    The peak intensity is a combination of the structure factor for a given
    peak and the position the Ewald sphere intersects the relrod. In this
    implementation, the intensity scales linearly with proximity.

    Parameters
    ----------
    structure : Structure
        The structure for which to derive the structure factors.
    indices : array-like
        The fractional coordinates of the peaks for which to calculate the
        structure factor.
    proximities : array-like
        The distances between the Ewald sphere and the peak centers.
    shape_factor_use : bool
        True if the shape factor correction needs to be used
    Returns
    -------
    peak_intensities : array-like
        The intensities of the peaks.

    """
    (
        coeffs,
        fcoords,
        occus,
        dwfactors,
    ) = get_vectorized_list_for_atomic_scattering_factors(
        structure=structure,
        debye_waller_factors=debye_waller_factors,
        scattering_params=scattering_params,
    )

    # Store array of g_hkls^2 values since used multiple times.

    ##length of the unique hkls
    g_hkls_sq = g_hkls_array ** 2

    # Create array containing atomic scattering factors.
    fs = get_atomic_scattering_factors(g_hkls_sq, coeffs, scattering_params)

    # Change the coordinate system of fcoords to align with that of g_indices
    fcoords = np.dot(
        fcoords,
        np.linalg.inv(np.dot(structure.lattice.stdbase, structure.lattice.recbase)),
    )

    # Calculate structure factors for all excited g-vectors.
    # dosnet like doing the dot on g_indices since is a dict keys not just array--qq usingunique_hkls here test!
    f_hkls = np.sum(
        fs
        * occus
        * np.exp(
            2j * np.pi * np.dot(g_indices, fcoords.T)
            - 0.25 * np.outer(g_hkls_sq, dwfactors)
        ),
        axis=-1,
    )

    # Define an intensity scaling that is linear with distance from Ewald sphere
    # along the beam direction.

    prefactor = shape_factor * multiplicites

    # Calculate the peak intensities from the structure factor and excitation
    peak_intensities = prefactor * (f_hkls * f_hkls.conjugate()).real
    return peak_intensities

In [ ]:
def calculate_profile_data(
        self,
        structure,
        reciprocal_radius=1.0,
        magnitude_tolerance=1e-5,
        minimum_intensity=1e-3,
    ):
        """
        Calculates a one dimensional diffraction profile for a structure.

        Parameters
        ----------
        structure : Structure
            The structure for which to calculate the diffraction profile.
        reciprocal_radius : float
            The maximum radius of the sphere of reciprocal space to sample, in
            reciprocal angstroms.
        magnitude_tolerance : float
            The minimum difference between diffraction magnitudes in reciprocal
            angstroms for two peaks to be consdiered different.
        minimum_intensity : float
            The minimum intensity required for a diffraction peak to be
            considered real. Deals with numerical precision issues.

        Returns
        -------
        diffsims.ProfileSimulation
            The diffraction profile corresponding to this structure and
            experimental conditions.
        """
        max_r = reciprocal_radius
        wavelength = self.wavelength
        scattering_params = self.scattering_params

        latt = structure.lattice
        is_hex = is_lattice_hexagonal(latt)

        # Obtain crystallographic reciprocal lattice points within range
        recip_latt = latt.reciprocal()
        spot_indices, _, spot_distances = get_points_in_sphere(
            recip_latt, reciprocal_radius
        )

        ##spot_indicies is a numpy.array of the hkls allowd in the recip radius
        unique_hkls, multiplicites, g_hkls = get_intensities_params(
            recip_latt, reciprocal_radius
        )
        g_indices = unique_hkls
        debye_waller_factors = self.debye_waller_factors
        excitation_error = None
        max_excitation_error = None
        g_hkls_array = np.asarray(g_hkls)

        i_hkl = get_kinematical_intensities(
            structure,
            g_indices,
            g_hkls_array,
            debye_waller_factors,
            multiplicites,
            scattering_params,
            shape_factor=1,
        )

        if is_hex:
            # Use Miller-Bravais indices for hexagonal lattices.
            g_indices = (g_indices[0], g_indices[1], -g_indices[0] - g_indices[1], g_indices[2])

        hkls_labels = ["".join([str(int(x)) for x in xs]) for xs in unique_hkls]

        peaks = {}
        for l, i, g in zip(hkls_labels, i_hkl, g_hkls):
            peaks[l] = [i, g]

        # Scale intensities so that the max intensity is 100.

        max_intensity = max([v[0] for v in peaks.values()])
        x = []
        y = []
        hkls = []
        for k in peaks.keys():
            v = peaks[k]
            if v[0] / max_intensity * 100 > minimum_intensity and (k != "000"):
                x.append(v[1])
                y.append(v[0])
                hkls.append(k)

        y = y / max(y) * 100

        return ProfileSimulation(x, y, hkls)


In [ ]:
def get_plot(self, g_max, annotate_peaks=True, with_labels=True, fontsize=12):

        """Plots the diffraction profile simulation for the
           calculate_profile_data method in DiffractionGenerator.

        Parameters
        ----------
        g_max : float
            Maximum g-vector magnitude to plot.
        annotate_peaks : boolean
            If True, peaks are annotaed with hkl information.
        with_labels : boolean
            If True, xlabels and ylabels are added to the plot.
        fontsize : integer
            Fontsize for peak labels.
        """

        ax = plt.gca()
        for g, i, hkls in zip(self.magnitudes, self.intensities, self.hkls):
            label = hkls
            ax.plot([g, g], [0, i], color="k", linewidth=3, label=label)
            if annotate_peaks:
                ax.annotate(label, xy=[g, i], xytext=[g, i], fontsize=fontsize)

            if with_labels:
                ax.set_xlabel("A ($^{-1}$)")
                ax.set_ylabel("Intensities (scaled)")

        return plt